In [1]:
import pandas as pd
import numpy as np
import os
import operator
import requests

In [130]:
df = pd.read_csv("Data/Yellowpages_20-04-2020.csv")
df2 = pd.read_csv("Data/Googlemaps_20-04-2020.csv")

In [131]:
df = pd.concat([df, df2], sort=False)

In [132]:
def merge_sheets(path = "Data/"):
    files = os.listdir(path)
    
    for file in files:
        initial = True
        if file[-3:] == "csv":
            if initial == True:
                df = pd.read_csv(path + file)
                initial = False
            else:
                this_df = pd.read_csv(path + file)
                df = pd.concat([df, this_df], sort=False)
    return df

def cleanup(df):
    df = df.drop(labels="Unnamed: 0", axis=1) 
            
    df = df.reset_index()
    df = df.drop(labels="index", axis=1)
    
    for i in range(0, len(df)):
        if df.iloc[i, 1][-11:] == ", Australia":
            df.iloc[i, 1] = df.iloc[i, 1][:-11]
            
        tel = str(df.Phone[i])
        bracket_o = tel.find("(")
        if bracket_o != -1:
            tel = tel[:bracket_o] + tel[bracket_o+2:]
            bracket_c = tel.find(")")
            if bracket_c != -1:
                tel = tel[:bracket_c] + tel[bracket_c+1:]
            if str(tel[:3]) != "+61":
                tel = "+61" + tel
        elif tel[0] == "0":
            tel = "+61" + tel[1:]
        df.Phone[i] = tel
            
    return df

def get_zip(df):
    zip_codes = list()
    state = list()
    for i in range(0, len(df.Name)):
        try:
            zip_code = df.Adress[i][-4:]
            zip_codes.append(zip_code)
        except:
            zip_codes.append("N/V")
            
        try:
            intzip_code = int(zip_code)
            if intzip_code >= 1000 and intzip_code < 3000:
                state.append("NSW")
            elif (intzip_code >= 3000 and intzip_code < 4000) or (intzip_code >= 8000 and intzip_code < 9000):
                state.append("Victoria")
            elif (intzip_code >= 4000 and intzip_code < 5000) or (intzip_code >= 9000 and intzip_code < 10000):
                state.append("Queensland")
            elif intzip_code >= 5000 and intzip_code < 6000:
                state.append("South Australia")
            elif intzip_code >= 6000 and intzip_code < 7000:
                state.append("Western Australia")
            elif intzip_code >= 7000 and intzip_code < 8000:
                state.append("Tasmania")
            elif intzip_code >= 800 and intzip_code < 1000:
                state.append("Nothern Australia")
            else:
                state.append("N/V")
        except:
            state.append("N/V")
            
            
    df["zip"] = zip_codes
    df["state"] = state
    return df

"""    for name in duplicates["names"]:
        part_df = df[df.Name == name]
        for i in range(0, len(part_df)):
            try:
                if len(part_df[part_df.zip == part_df.zip[i]]) > 1:
                    print("zip here.")
            except:
                pass"""

def find_duplicates(df):
    cutOff = int((len(df)/2) + 1)
    duplicates = list()
    for i in range(0, cutOff):
        if len(df[df.Name == df.Name[i]]) > 1:
            duplicates.append(df.Name[i])
    
    return duplicates

def handle_duplicates(df, duplicates):
    columns = ["Adress", "GeoLocation", "Phone", "Website", "Email"]
    for i in range(0, len(duplicates)):
        part_df = df[(df.Name == df.Name[i]) & (df.zip == df.zip[i])]

        best_entry = dict()
        for column in columns:
            for i, entry in enumerate(part_df[column]):
                   if entry != "N/V" and entry != "":
                        if i not in best_entry.keys():
                            best_entry[i] = 1
                        else:
                            best_entry[i] = best_entry[i] + 1

        best_entry = max(best_entry.items(), key=operator.itemgetter(1))[0]
        empty_columns = list()


        for column in columns:
            try:
                if part_df[column][best_entry] == "N/V":
                    empty_columns.append(column)
            except:
                pass

        if len(empty_columns) != 0:
            for i in range(0, len(part_df)):
                if i != best_entry:
                    for column in empty_columns:
                        if part_df[column][i] != "N/V" and part_df[column][i] != "":
                              part_df[column][best_entry] =  part_df[column][i]

        df = df.drop(index=part_df.index)
        df = df.append(part_df.iloc[best_entry])
    return df

In [133]:
#df = merge_sheets()

In [134]:
df = cleanup(df)

In [135]:
df = get_zip(df)

In [23]:
duplicates = find_duplicates(df)

In [136]:
df.head()

,Name,Adress,GeoLocation,Phone,Website,Email,Source,DateUpdated,DateChecked,zip,state
0,The Farm Byron Bay,"11 Ewingsdale Rd, Ewingsdale NSW 2481",NaN,+612 6684 7888,http://www.thefarmbyronbay.com.au/,info@thefarmbyronbay.com.au,https://www.yellowpages.com.au/search/listings...,20/04/2020,-,2481,NSW
1,GCS Partners In Farming,"15 / 553 New Grenfell Rd, Forbes NSW 2871",NaN,+61409 364 382,N/V,gcmoxey@bigpond.com,https://www.yellowpages.com.au/search/listings...,20/04/2020,-,2871,NSW
2,Boer Bok Goat Stud,"PO Box 537, Busselton WA 6280",NaN,+618 9753 1160,http://www.boerbok.com.au/,bethh@boerbok.net.au,https://www.yellowpages.com.au/search/listings...,20/04/2020,-,6280,Western Australia
3,Kilsby Malcolm & Karen,"PO Box 458, Mt Gambier SA 5290",NaN,+618 8738 5322,N/V,N/V,https://www.yellowpages.com.au/search/listings...,20/04/2020,-,5290,South Australia
4,Baboo Pastoral Co,"""Wandija"", Green Range WA 6328",NaN,+618 9846 6031,N/V,N/V,https://www.yellowpages.com.au/search/listings...,20/04/2020,-,6328,Western Australia


In [10]:
len(df)

1495

In [11]:
df_noDuplicates = handle_duplicates(df, duplicates)

In [12]:
df_noDuplicates.head()

,Name,Adress,GeoLocation,Phone,Website,Email,Source,DateUpdated,DateChecked,zip,state
43,Babylon Farm,Murrumbateman ACT 2582,NaN,0477 330 598,http://www.babylonfarm.com.au/,visit@babylonfarm.com.au,https://www.yellowpages.com.au/search/listings...,20/04/2020,-,2582,NSW
44,Care Outreach,"35 Dacmar Rd, Coolum Beach QLD 4573",NaN,(07) 5471 7636,https://careoutreach.com.au/,info@careoutreach.com.au,https://www.yellowpages.com.au/search/listings...,20/04/2020,-,4573,Queensland
45,Biomix,"980 Two Tree Rd, Carag Carag VIC 3623",NaN,0488 993 066,https://www.biomix.com.au/,info@biomix.com.au,https://www.yellowpages.com.au/search/listings...,20/04/2020,-,3623,Victoria
55,K S Benning & Sons,"24B Bosworth Rd, Woolgoolga NSW 2456",NaN,(02) 6654 8775,http://www.ksbenning.com.au/,admin@ksbenning.com.au,https://www.yellowpages.com.au/search/listings...,20/04/2020,-,2456,NSW
56,CN & BL McGillivray,N/V,NaN,(03) 5593 2967,http://www.cnandblmcgillivray.com.au/,col.bev3260@gmail.com,https://www.yellowpages.com.au/search/listings...,20/04/2020,-,N/V,N/V


In [13]:
def try_websites(df):
    for i in range(0, len(df.Name)):
        if i % 100 == 0:
            print(i)
        try:
            request = requests.get(str(df.Website[i]))
        except:
            df.Website[i] = "N/V"
    return df